In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install einops
!pip install peft
!pip install trl
!pip install bitsandbytes
!pip install datasets==2.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling 

In [2]:
import os
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import json
import warnings

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer
from huggingface_hub import interpreter_login
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore")

2024-04-21 18:49:08.374505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 18:49:08.374611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 18:49:08.495451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
result = list(df.to_json(orient="records"))

result[0] = '{"json":['
result[-1] = ']'
result.append('}')


result = ''.join(result)
result = result.strip('"\'')

result = json.loads(result)

with open('data.json', 'w') as json_file:
    json.dump(result, json_file)

In [7]:
df = pd.read_csv("/kaggle/input/medquad/medquad.csv")
df = df.iloc[:,:2]
df.columns = ["text",'label']
df.head()

,text,label
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea..."
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ..."
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...


In [8]:
def formatting_func(example):
    text = f"### Question: {example['text']}\n ### Answer: {example['label']}"
    return text

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [18]:
def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device_map()  # 'cpu'

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)


#Download model
model = AutoModelForCausalLM.from_pretrained(
        "microsoft/phi-2", 
        #quantization_config=bnb_config, 
        device_map = device,
        trust_remote_code=True,
        use_auth_token=False
#         offload_folder=huggingface_params.offload_folder
    )

model.config.pretraining_tp = 1 
peft_config = LoraConfig(
r=32,
lora_alpha=16,
bias="none",
lora_dropout=0.05, # Conventional
task_type="CAUSAL_LM",

)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True, padding='max_length',truncation=True,  return_tensors="pt")
tokenizer.pad_token = tokenizer.eos_token

# model.config.save_pretrained(huggingface_params.offload_folder)
# model.save_pretrained(huggingface_params.offload_folder)

training_arguments = TrainingArguments(
output_dir= "./results",
num_train_epochs= 4,
per_device_train_batch_size= 1,
gradient_accumulation_steps= 8,
save_steps = 0,
optim="paged_adamw_32bit",
save_strategy="epoch",
logging_steps=100,
logging_strategy="steps",
learning_rate= 2e-4,
fp16= True,
bf16= False,
group_by_length= True,
lr_scheduler_type="cosine",
disable_tqdm=False,
report_to=None,
evaluation_strategy="epoch"
)
model.config.use_cache = False

dataset = load_dataset("json", data_files="/kaggle/working/data.json", field='json', split="train")
dataset = dataset.map(generate_and_tokenize_prompt)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 38.12 MiB is free. Process 7329 has 15.86 GiB memory in use. Of the allocated memory 15.59 GiB is allocated by PyTorch, and 15.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pipreqs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 14.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: ipython
    Found existing installation: ipython 8.20.0
    Uninstalling ipython-8.20.0:
      Successfully uninstalled ipython-8.20.0
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.4.5
    Uninstalling nbconvert-6.4.5:
      Successfully uninstalled nbconvert-6.4.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatibl

In [25]:
pipreqsnb . 

SyntaxError: invalid syntax (2345228751.py, line 1)

In [26]:
import os

# Define the path for the requirements file
requirements_path = "/kaggle/working/requirements.txt"

# Use pip freeze to capture all installed packages and their versions
# Write the output to the requirements.txt file
with open(requirements_path, "w") as file:
    # Execute pip freeze and write the output to file
    os.system(f"pip freeze > {requirements_path}")

# Optional: Print the contents of requirements.txt to verify
with open(requirements_path, "r") as file:
    print(file.read())

absl-py==1.4.0
accelerate==0.28.0
access==1.1.9
affine==2.4.0
aiobotocore==2.12.2
aiofiles==22.1.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1701099469104/work
aiohttp-cors==0.7.0
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
aiosqlite==0.19.0
albumentations==1.4.0
alembic==1.13.1
altair==5.3.0
annotated-types @ file:///home/conda/feedstock_root/build_artifacts/annotated-types_1696634205638/work
annoy==1.17.3
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1702909220329/work
apache-beam==2.46.0
aplus==0.11.0
appdirs==1.4.4
archspec @ file:///home/conda/feedstock_root/build_artifacts/archspec_1708969572489/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386546427/work
array-record==0.5.0
arrow @ file:///home/conda/fe